Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/deployment/production-deploy-to-aks/production-deploy-to-aks.png)

# Deploy models to Azure Kubernetes Service (AKS) using controlled roll out
This notebook will show you how to deploy mulitple AKS webservices with the same scoring endpoint and how to roll out your models in a controlled manner by configuring % of scoring traffic going to each webservice. If you are using a Notebook VM, you are all set. Otherwise, go through the [configuration notebook](../../../configuration.ipynb) to install the Azure Machine Learning Python SDK and create an Azure ML Workspace.

In [ ]:
# Check for latest version
import azureml.core
print(azureml.core.VERSION)

## Initialize workspace
Create a [Workspace](https://docs.microsoft.com/python/api/azureml-core/azureml.core.workspace%28class%29?view=azure-ml-py) object from your persisted configuration.

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Register the model
Register a file or folder as a model by calling [Model.register()](https://docs.microsoft.com/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#register-workspace--model-path--model-name--tags-none--properties-none--description-none--datasets-none--model-framework-none--model-framework-version-none--child-paths-none-).
In addition to the content of the model file itself, your registered model will also store model metadata -- model description, tags, and framework information -- that will be useful when managing and deploying models in your workspace. Using tags, for instance, you can categorize your models and apply filters when listing models in your workspace.

In [ ]:
from azureml.core import Model

model = Model.register(workspace=ws,
                       model_name='sklearn_regression_model.pkl',                # Name of the registered model in your workspace.
                       model_path='./sklearn_regression_model.pkl',  # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       model_framework_version='0.19.1',             # Version of scikit-learn used to create the model.
                       description='Ridge regression model to predict diabetes progression.',
                       tags={'area': 'diabetes', 'type': 'regression'})

print('Name:', model.name)
print('Version:', model.version)

## Register an environment (for all models)

If you control over how your model is run, or if it has special runtime requirements, you can specify your own environment and scoring method.

Specify the model's runtime environment by creating an [Environment](https://docs.microsoft.com/python/api/azureml-core/azureml.core.environment%28class%29?view=azure-ml-py) object and providing the [CondaDependencies](https://docs.microsoft.com/python/api/azureml-core/azureml.core.conda_dependencies.condadependencies?view=azure-ml-py) needed by your model.

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

environment=Environment('my-sklearn-environment')
environment.python.conda_dependencies = CondaDependencies.create(conda_packages=[
    'pip==20.2.4'],
    pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'numpy==1.23',
    'scikit-learn==0.22.1',
    'scipy'
])

When using a custom environment, you must also provide Python code for initializing and running your model. An example script is included with this notebook.

In [ ]:
with open('score.py') as f:
    print(f.read())

## Create the InferenceConfig
Create the inference configuration to reference your environment and entry script during deployment

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='score.py', 
                                   source_directory='.',
                                   environment=environment)


## Provision the AKS Cluster
If you already have an AKS cluster attached to this workspace, skip the step below and provide the name of the cluster.

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

In [ ]:
from azureml.core.compute import AksCompute
from azureml.core.compute import ComputeTarget
# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration()

aks_name = 'my-aks' 
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config) 
aks_target.wait_for_completion(show_output=True)

## Create an Endpoint and add a version (AKS service)
This creates a new endpoint and adds a version behind it. By default the first version added is the default version. You can specify the traffic percentile a version takes behind an endpoint. 


In [ ]:
# deploying the model and create a new endpoint
from azureml.core.webservice import AksEndpoint
# from azureml.core.compute import ComputeTarget

#select a created compute
compute = ComputeTarget(ws, 'my-aks')
namespace_name="endpointnamespace"
# define the endpoint name
endpoint_name = "myendpoint1"
# define the service name
version_name= "versiona"

endpoint_deployment_config = AksEndpoint.deploy_configuration(tags = {'modelVersion':'firstversion', 'department':'finance'}, 
                                                              description = "my first version", namespace = namespace_name, 
                                                              version_name = version_name, traffic_percentile = 40)

endpoint = Model.deploy(ws, endpoint_name, [model], inference_config, endpoint_deployment_config, compute)
endpoint.wait_for_deployment(True)

In [ ]:
endpoint.get_logs()

## Add another version of the service to an existing endpoint
This adds another version behind an existing endpoint. You can specify the traffic percentile the new version takes. If no traffic_percentile is specified then it defaults to 0. All the unspecified traffic percentile (in this example 50) across all versions goes to default version.

In [ ]:
# Adding a new version to an existing Endpoint.
version_name_add="versionb" 

endpoint.create_version(version_name = version_name_add, inference_config=inference_config, models=[model], tags = {'modelVersion':'secondversion', 'department':'finance'}, 
                        description = "my second version", traffic_percentile = 10)
endpoint.wait_for_deployment(True)

## Update an existing version in an endpoint
There are two types of versions: control and treatment. An endpoint contains one or more treatment versions but only one control version. This categorization helps compare the different versions against the defined control version.

In [ ]:
endpoint.update_version(version_name=endpoint.versions[version_name_add].name, description="my second version update", traffic_percentile=40, is_default=True, is_control_version_type=True)
endpoint.wait_for_deployment(True)

## Test the web service using run method
Test the web sevice by passing in data. Run() method retrieves API keys behind the scenes to make sure that call is authenticated.

In [ ]:
# Scoring on endpoint
import json
test_sample = json.dumps({'data': [
    [1,2,3,4,5,6,7,8,9,10], 
    [10,9,8,7,6,5,4,3,2,1]
]})

test_sample_encoded = bytes(test_sample, encoding='utf8')
prediction = endpoint.run(input_data=test_sample_encoded)
print(prediction)

## Delete Resources

In [ ]:
# deleting a version in an endpoint
endpoint.delete_version(version_name=version_name)
endpoint.wait_for_deployment(True)

In [ ]:
# deleting an endpoint, this will delete all versions in the endpoint and the endpoint itself
endpoint.delete()